In [ ]:
import pandas as pd
from surprise import Reader, Dataset, KNNBasic, SVD, NMF
import sys
sys.path.insert(0, '../models')
from CrossVal import get_cross_validation
sys.path.insert(0, '../benchmark')
from evaluateCF import find_best_params, get_ndcg

In [13]:
df_combined = pd.read_csv('../data/interim/train_cleaned.csv')
df_combined

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,composer,...,gender,registered_via,registration_init_time,expiration_date,registration_init_time_year,registration_init_time_month,registration_init_time_day,expiration_date_year,expiration_date_month,expiration_date_day
0,24088,164688,3,8,4,0,279264.0,318,32116,53254,...,2,4,3540,415,2016,3,9,2016,3,12
1,3793,168169,0,4,6,1,205224.0,318,25000,39784,...,0,3,3725,872,2016,11,13,2017,7,14
2,28638,228433,6,15,8,0,251559.0,313,455,205,...,2,7,1918,716,2011,6,9,2017,2,8
3,5238,219627,0,4,6,1,201084.0,313,8591,13594,...,0,9,440,950,2006,5,11,2017,9,30
4,7131,83427,3,8,3,1,302881.0,173,18811,49045,...,0,9,389,969,2006,3,3,2017,10,19


Создание матрицы элементов пользователя

In [14]:
util_mat = df_combined.pivot_table(index = 'msno', columns = 'song_id', values = 'target')
util_mat.head(20)

song_id,10,18,21,32,36,58,68,71,72,76,...,260028,260041,260048,260050,260057,260058,260060,260064,260096,260098
msno,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Создание коллаборативной рекомендательной системы на основе KNN

In [16]:
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df( df_combined[['msno', 'song_id', 'target']], reader = reader )

In [17]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=.2)

In [19]:
# Compute Mean Squared Distance Similarity
sim_options = {'name' : 'msd'}

algo = KNNBasic(k=20, sim_options=sim_options )
get_cross_validation(algo, data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.5033  0.5039  0.5033  0.5035  0.0003  
Fit time          7.85    7.90    8.23    7.99    0.17    
Test time         0.83    0.78    0.93    0.85    0.06    


In [21]:
n_neighbours = [5, 10, 20, 30]
n_epochs = [5, 10, 20]

param_grid = {'n_neighbours' : n_neighbours, 'n_epochs' : n_epochs}


best_score, best_params = find_best_params(data, param_grid, KNNBasic)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [22]:
# Best RMSE score
print('Best Score :', best_score)

# Combination of parameters that gave the best RMSE score
print('Best Parameters :', best_params)

Best Score : 0.504994147375943
Best Parameters : {'n_neighbours': 5, 'n_epochs': 5}


In [23]:
# Train the model with best parameters
best_k = best_params['n_neighbours'] 
best_epochs = best_params['n_epochs']
algo = KNNBasic(k=best_k, sim_options={'name': 'msd'}, verbose=True)
algo.fit(trainset)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [24]:
# Make predictions on the test set
predictions = algo.test(testset)

from surprise import accuracy

rmse = accuracy.rmse(predictions)
print(f'RMSE: {rmse:.4f}')


RMSE: 0.5062
RMSE: 0.5062


In [25]:
ndcg_at_20 = get_ndcg(predictions, k_highest_scores=20)
print(f'NDCG@20: {ndcg_at_20:.4f}')


NDCG@20: 0.6378
